## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## 在我們開始之前 - 一些設定:


請造訪 Sendgrid 網站：https://sendgrid.com/

(Sendgrid 是一家 Twilio 公司，用於發送電子郵件。)

如果 SendGrid 出現問題，請參考替代實現，使用 "Resend Email" 位於 community_contributions/2_lab2_with_resend_email

請建立一個帳戶 - 現在是免費的！（至少對我來說，目前是免費的）。

建立帳戶後，點擊：

設定（左側邊欄）>> API Keys >> 建立 API Key（右上角按鈕）

將金鑰複製到剪貼簿，然後新增一行到您的 .env 檔案：

`SENDGRID_API_KEY=xxxx`

此外，在 SendGrid 中，前往：

設定（左側邊欄）>> 發件人驗證 >> "Verify a Single Sender"  
並驗證您的電子郵件地址是一個真實的電子郵件地址，以便 SendGrid 可以為您發送電子郵件。



In [10]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [11]:
load_dotenv(override=True)

True

In [12]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("sacahan@gmail.com")  # Change to your verified sender
    to_email = To("sacahan@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

ForbiddenError: HTTP Error 403: Forbidden

### Did you receive the test email

If you get a 202, then you're good to go!

#### Certificate error

If you get an error SSL: CERTIFICATE_VERIFY_FAILED then students Chris S and Oleksandr K have suggestions:  
First run this: `!uv pip install --upgrade certifi`  
Next, run this:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

#### Other errors or no email

If there are other problems, you'll need to check your API key and your verified sender email address in the SendGrid dashboard

Or use the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

(Or - you could always replace the email sending code below with a Pushover call, or something to simply write to a flat file)

In [7]:
!uv pip install --upgrade certifi



Using Python 3.12.11 environment at: /Users/sacahan/Documents/workspace/agents/.venv
Resolved 1 package in 198ms                                          
Prepared 1 package in 1ms                                                
Uninstalled 1 package in 3ms
Installed 1 package in 2ms                                  
 - certifi==2025.6.15
 + certifi==2025.8.3


In [8]:
import certifi
import os

os.environ["SSL_CERT_FILE"] = certifi.where()


## Step 1: Agent workflow

In [13]:
instructions1 = "您是一名為 ComplAI 工作的銷售代理，\
ComplAI 是一家提供由 AI 驅動的 SOC2 合規性保障和審計準備的 SaaS 工具的公司。\
您撰寫專業且嚴肅的冷郵件。"

instructions2 = "您是一名幽默且具有吸引力的銷售代理，為 ComplAI 工作，\
ComplAI 是一家提供由 AI 驅動的 SOC2 合規性保障和審計準備的 SaaS 工具的公司。\
您撰寫風趣且吸引人的冷郵件，能夠提高回覆率。"

instructions3 = "您是一名忙碌的銷售代理，為 ComplAI 工作，\
ComplAI 是一家提供由 AI 驅動的 SOC2 合規性保障和審計準備的 SaaS 工具的公司。\
您撰寫簡潔明瞭的冷郵件。"

In [14]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:
# 使用 sales_agent1 以繁體中文撰寫冷郵件
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email using zh_tw")

# 非同步處理事件流，逐步輸出生成的內容
async for event in result.stream_events():
    # 檢查事件類型是否為原始回應事件，並且資料類型為 ResponseTextDeltaEvent
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        # 輸出生成的文字內容，並即時刷新輸出
        print(event.data.delta, end="", flush=True)

主旨：提升您的 SOC2 合規性保障與審計準備

尊敬的 [收件人姓名]，

您好！我希望這封郵件能夠為您帶來一些值得關注的資訊。

在當前數位化加速的時代，企業面對的安全挑戰日益增加，SOC2 合規性已成為確保客戶信任與資料安全的關鍵。然而，合規過程往往繁瑣且耗時，給企業帶來了不小的壓力。

作為 ComplAI 的銷售代理，我們的目標是幫助企業簡化這一過程。我們的 AI 驅動解決方案不僅能夠自動化合規性檢查，還能提供實時的審計準備支持。這樣一來，您可以專注於業務增長，而無需過度擔憂合規性問題。

我們的工具具備以下幾項優勢：

- 自動化的 SOC2 評估，節省時間與資源
- 實時的合規性報告，提高透明度
- 專業的客戶支持，確保您的需求得到滿足

如果您有興趣了解更多，或是希望安排一次示範，請隨時回覆此郵件。我們非常期待能有機會與您進一步探討。

謝謝您的時間，期待您的回覆。

祝好，

[您的姓名]  
[您的職位]  
ComplAI  
[您的聯繫方式]  
[公司網站]  

In [19]:
message = "Write a cold sales email using zh_tw"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n----\n\n")


主旨：提升您的 SOC2 合規性保障與審計準備

尊敬的 [收件人姓名]，

您好！我希望這封郵件能夠引起您的注意。我是來自 ComplAI 的 [您的名字]，我們提供一款由 AI 驅動的 SaaS 工具，專門協助企業簡化 SOC2 合規性保障與審計準備的過程。

在如今數據安全與隱私日益重要的時代，確保 SOC2 合規性不僅是法律要求，更是贏得客戶信任的關鍵。使用我們的工具，您可以：

- 自動化合規性檢查，減少人工錯誤
- 快速生成所需文檔，縮短審計準備時間
- 實時監控和風險評估，及時識別潛在問題

我們的客戶已經在使用 ComplAI 後見證了顯著的效率提升與成本節省。我們相信，您也能從中受益。

如果您對提升貴公司的 SOC2 合規性感興趣，請隨時回覆此郵件或安排一次簡短的會議，我將非常樂意進一步介紹我們的解決方案。

感謝您的時間，期待您的回覆！

祝好，

[您的名字]  
[您的職位]  
ComplAI  
[您的聯繫方式]  
[公司網站]  
----


主旨：讓您的合規性變得輕鬆有趣！

親愛的 [客戶姓名]，

您好！希望這封郵件能像一杯咖啡一樣讓您倍感振奮！☕️

當您聽到「SOC2 合規性」這四個字時，是不是已經感受到那股來自繁瑣文件的恐懼感了呢？別擔心，我們有一個解決方案能讓您擺脫這種壓力——ComplAI！我們的 AI 驅動平台不僅能幫助您輕鬆準備審計，還能使整個過程變得更有趣（相信我們，這不是玩笑！）。

想像一下，您只需幾次點擊，就能讓合規性的繁瑣工作化為烏有！我們的工具會為您提供清晰的指導和支援，讓您誠惶誠恐的合規之路變得輕鬆自在，甚至還有時間喝杯咖啡（或兩杯？）。

想要進一步了解我們的解決方案如何拯救您的合規性之旅嗎？我很樂意安排一次簡短的通話，讓它成為您一天中最有趣的那段時間！

期待您的回覆，讓我們一起攜手迎接合規性的挑戰！

最佳祝福，

[您的名字]  
ComplAI 團隊  
[您的聯絡方式]  
[公司網站]  

P.S. 我們對合規的熱情，讓我們相信自己是市場上最酷的工具之一！😄
----


主旨：提升您的 SOC2 合規性保障

親愛的 [收件人名稱]，

希望您一切安好！我是來自 ComplAI 的 [您的名字]，我們專注於提供由 AI 驅動的 SOC2 合規性保障和審計準備工具。

隨著合規需求

In [20]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [21]:
message = "Write a cold sales email using zh_tw"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
主旨：讓您的 SOC2 合規更輕鬆——無需裝飾的 AI 魔法！

尊敬的 [客戶姓名]，

希望這封郵件能讓您的一天更加光彩！🌟 我是 ComplAI 的 [您的名字]，專門幫助企業像您的企業一樣，乘風破浪，迎接 SOC2 合規的挑戰。

您可能會想：“社會合規？這不是我夢想中的度假計劃！”😅 但別擔心，我們的 AI 驅動工具，就像是專業的潛水教練，能夠引導您在合規海洋中暢游，而不會讓您感覺像在沙灘上撿垃圾。

我們的系統能夠：
1. 自動化合規流程，讓您擺脫繁瑣的文書工作。
2. 提供即時的審計準備，讓您可以像準備好上台演出那樣，信心滿滿！
3. 大幅降低成本，讓您的財務部門歡呼不已！

如果您想了解更多，或者想找個理由來喝杯咖啡，我隨時可以安排一次簡短的通話。畢竟，您值得擁有最輕鬆的合規經歷！

期待您的回覆，無論是好消息或是想聊聊！😄

最佳祝福，

[您的名字]  
[您的職位]  
ComplAI  
[您的電話]  
[您的電子郵件]  
[網站鏈接]  

P.S. 如果您今天的目標是讓合規變得輕鬆有趣，那我們的工具絕對是您的最佳選擇！🎉


Now go and check out the trace:

https://platform.openai.com/traces

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [22]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [23]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='您是一名為 ComplAI 工作的銷售代理，ComplAI 是一家提供由 AI 驅動的 SOC2 合規性保障和審計準備的 SaaS 工具的公司。您撰寫專業且嚴肅的冷郵件。', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [ ]:
@function_tool # 定義為發送電子郵件的工具
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [25]:
# Let's look at it
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1080925c0>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [26]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10aa9b920>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [ ]:
description = "Write a cold sales email"

# 將Agent定義為工具
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10aa9ba60>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10aa9b880>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required'

## And now it's time for our Sales Manager - our planning agent

In [ ]:
# Improved instructions thanks to student Guillermo F.

instructions = """
您是 ComplAI 的銷售經理。您的目標是使用 sales_agent 工具找到單一最佳的冷銷售郵件。

請仔細遵循以下步驟：
1. 生成草稿：使用所有三個 sales_agent 工具生成三個不同的郵件草稿。在所有三個草稿準備好之前，請勿繼續。

2. 評估並選擇：審查草稿並根據您對哪一封最有效的判斷，選擇單一最佳郵件。

3. 使用 send_email 工具發送最佳郵件（且僅發送最佳郵件）給用戶。

重要規則：
- 您必須使用 sales_agent 工具生成草稿——請勿自行撰寫。
- 您必須使用 send_email 工具發送一封郵件——絕不能超過一封。
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Wait - you didn't get an email??</h2>
            <span style="color:#ff7800;">With much thanks to student Chris S. for describing his issue and fixes. 
            If you don't receive an email after running the prior cell, here are some things to check: <br/>
            First, check your Spam folder! Several students have missed that the emails arrived in Spam!<br/>Second, print(result) and see if you are receiving errors about SSL. 
            If you're receiving SSL errors, then please check out theses <a href="https://chatgpt.com/share/680620ec-3b30-8012-8c26-ca86693d0e3d">networking tips</a> and see the note in the next cell. Also look at the trace in OpenAI, and investigate on the SendGrid website, to hunt for clues. Let me know if I can help!
            </span>
        </td>
    </tr>
</table>

### And one more suggestion to send emails from student Oleksandr on Windows 11:

If you are getting certificate SSL errors, then:  
Run this in a terminal: `uv pip install --upgrade certifi`

Then run this code:
```python
import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()
```

Thank you Oleksandr!

## Remember to check the trace

https://platform.openai.com/traces

And then check your email!!


### Handoffs 代表代理可以將控制權委派給另一個代理的方式

Handoffs 和 Agents-as-tools 是相似的：

在這兩種情況下，一個代理都可以與另一個代理合作

使用工具時，控制權會返回

使用 Handoffs 時，控制權會轉移


In [28]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [30]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
# 撰寫 email subject ，HTML 轉換工具與發送電子郵件工具
tools = [subject_tool, html_tool, send_html_email]

In [32]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ab94040>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10ab94180>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales pros

In [ ]:
instructions = "您是一名電子郵件格式化和發送專家。您將收到一封待發送的電子郵件正文。\
您首先使用 subject_writer 工具為該郵件撰寫主旨，然後使用 html_converter 工具將正文轉換為 HTML 格式。\
最後，您使用 send_html_email 工具，將該郵件連同主旨和 HTML 正文一起發送。"


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


### Now we have 3 tools and 1 handoff

In [ ]:
tools = [tool1, tool2, tool3] # 三種mail風格撰寫工具
handoffs = [emailer_agent]
print(tools)
print(handoffs)

In [ ]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
您是 ComplAI 的銷售經理。您的目標是使用 sales_agent 工具找到單一最佳的冷銷售郵件。

請仔細遵循以下步驟：
1. 生成草稿：使用所有三個 sales_agent 工具生成三個不同的郵件草稿。在所有三個草稿準備好之前，請勿繼續。

2. 評估並選擇：審查草稿並根據您對哪一封最有效的判斷，選擇單一最佳郵件。
您可以多次使用工具，如果對第一次的結果不滿意。

3. 移交發送：僅將獲勝的郵件草稿移交給 "Email Manager" 代理。Email Manager 將負責格式化和發送。

重要規則：
- 您必須使用 sales_agent 工具生成草稿——請勿自行撰寫。
- 您必須僅移交一封郵件給 Email Manager——絕不能超過一封。
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

### Remember to check the trace

https://platform.openai.com/traces

And then check your email!!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Can you identify the Agentic design patterns that were used here?<br/>
            What is the 1 line that changed this from being an Agentic "workflow" to "agent" under Anthropic's definition?<br/>
            Try adding in more tools and Agents! You could have tools that handle the mail merge to send to a list.<br/><br/>
            HARD CHALLENGE: research how you can have SendGrid call a Callback webhook when a user replies to an email,
            Then have the SDR respond to keep the conversation going! This may require some "vibe coding" 😂
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">This is immediately applicable to Sales Automation; but more generally this could be applied to  end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution
            like this in your day job.
            </span>
        </td>
    </tr>
</table>

## Extra note:

Google 已經發布了他們的 Agent Development Kit (ADK)。雖然目前還沒有像本課程中的其他框架那樣受到廣泛關注，但它正在逐漸引起一些注意。有趣的是，它看起來與 OpenAI Agents SDK 非常相似。為了讓您一睹為快，以下是 ADK 的範例程式碼：

```
root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time]
)
```

Well, that looks familiar!

And a student has contributed a customer care agent in community_contributions that uses ADK.